# Example State Machine Usage



In [1]:
from state_machine import StateMachine, State, Model, Condition, AlwaysTrue, AlwaysFalse

In [2]:
class ReceptionistModel( Model ):
    def __init__( self, name='Bernie' ):
        super( ReceptionistModel, self ).__init__( name )
        
model = ReceptionistModel()

In [3]:
class PhoneRings(Condition):
    def __init__(self):
        self.name='Phone Rings'
    def is_triggered(self, data, model):
        return data=='ring ring'
PHONE_RINGS = PhoneRings()
assert PHONE_RINGS.is_triggered( 'ring ring', None )==True
assert PHONE_RINGS.is_triggered( 'silence', None )==False

In [4]:
class CustomerWants(Condition):
    def __init__(self, wants):
        self.name='Wants %s'%wants
        self.wants=wants
        
    def is_triggered(self, data, model):
        if isinstance(data, dict):
            return self.wants==data.get('customer_wants', self.wants)

CUSTOMER_WANTS_PIZZA   = CustomerWants('pizza')
CUSTOMER_WANTS_LASAGNE = CustomerWants('lasagne')

assert CUSTOMER_WANTS_PIZZA.is_triggered( {'customer_wants':'lasagne'}, model )==False
assert CUSTOMER_WANTS_PIZZA.is_triggered( {'customer_wants':'pizza'}, model )==True

In [5]:
receptionist = StateMachine( 'Receptionist' )

wait_for_phone    = State('Wait For Phone')
take_order        = State('Take Order')

enter_lasagne_order = State( 'Enter Lasagne Order' )
enter_pizza_order   = State('Enter Pizza Order' )

wait_for_phone.add_transition_to(take_order,PHONE_RINGS)
take_order.add_transition_to( enter_lasagne_order, CUSTOMER_WANTS_LASAGNE )
take_order.add_transition_to( enter_pizza_order, CUSTOMER_WANTS_PIZZA )

receptionist.add_state( wait_for_phone )
receptionist.add_state( take_order )

model.set_state( wait_for_phone )

In [6]:
receptionist.run( 'SILENCE', model )

INFO:default:Testing Transition from State:Wait For Phone to State:Take Order, Conditions:(Phone Rings)


In [7]:
receptionist.run('ring ring', model)
print(model.current_state)

INFO:default:Testing Transition from State:Wait For Phone to State:Take Order, Conditions:(Phone Rings)
INFO:default:Transition from State:Wait For Phone to State:Take Order triggered
INFO:default:Testing Transition from State:Take Order to State:Enter Lasagne Order, Conditions:(Wants lasagne)
INFO:default:Testing Transition from State:Take Order to State:Enter Pizza Order, Conditions:(Wants pizza)


State:Take Order


In [8]:
receptionist.run({'customer_wants':'pizza'},model)
print( model.current_state )

INFO:default:Testing Transition from State:Take Order to State:Enter Lasagne Order, Conditions:(Wants lasagne)
INFO:default:Testing Transition from State:Take Order to State:Enter Pizza Order, Conditions:(Wants pizza)
INFO:default:Transition from State:Take Order to State:Enter Pizza Order triggered


State:Enter Pizza Order


In [9]:
model.set_state( wait_for_phone )
receptionist.run( 'SILENCE', model )
receptionist.run('ring ring', model)
receptionist.run({'customer_wants':'lasagne'},model)
print( model.current_state )

INFO:default:Testing Transition from State:Wait For Phone to State:Take Order, Conditions:(Phone Rings)
INFO:default:Testing Transition from State:Wait For Phone to State:Take Order, Conditions:(Phone Rings)
INFO:default:Transition from State:Wait For Phone to State:Take Order triggered
INFO:default:Testing Transition from State:Take Order to State:Enter Lasagne Order, Conditions:(Wants lasagne)
INFO:default:Testing Transition from State:Take Order to State:Enter Pizza Order, Conditions:(Wants pizza)
INFO:default:Testing Transition from State:Take Order to State:Enter Lasagne Order, Conditions:(Wants lasagne)
INFO:default:Transition from State:Take Order to State:Enter Lasagne Order triggered


State:Enter Lasagne Order
